In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as sstat
from mpmath import mp
import math
import random

psd = pd.DataFrame
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [2]:
def single_lik(mu):
   y= tfd.MultivariateNormalFullCovariance(mu,sigma_obs)
   return y

In [ ]:
## mu_obs=np.array([1,4])
## mu1=[1,4]
## sigma_obs=np.array([[1,0.5],[0.5,1]])
## sigma1=[[1, 0.5],[0.5,1]]
## y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)
## y_obs

In [5]:
def prior():
    return tfd.MultivariateNormalFullCovariance(mu0,sigma0)

In [39]:
def proposal(theta):
  return tfd.MultivariateNormalFullCovariance(theta,cov)

In [120]:
def K_h(u,H):
    return 1/(np.sqrt(2*math.pi))*pow(np.linalg.det(H), -1/2)*np.exp(-1/2*u@np.linalg.inv(H)@u.T)

In [124]:
u= np.array([[1.432,1.1674]])
H= np.array([[1,0],[0,1]])
K_h(u,H)

array([[0.07239182]])

In [ ]:
def S1(y):
  quantiles=[]
  for i in range(9):
      quantiles.append(np.quantile(y,0.1+0.1*i))         #quantili 0,1 0,2 0,3 0,4 0,5 0,6 0,7 0,8 0,9
  return quantiles

In [30]:
def S2(y):
  return np.mean(y,axis=0)


In [28]:
mu_obs = np.array ([[1,4]])
sigma_obs=np.array([[1,0.5],[0.5,1]])
y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)
np.mean(y_obs,axis=0)

array([[0.940725  , 3.92846416]])

In [35]:
S2(y_obs)

AttributeError: ignored

In [126]:
s_0 = S2(single_lik(prior().sample()).sample(1000))
s_obs = S2(y_obs)
print(s_0.shape, s_obs.shape)
np.linalg.norm(np.array(s_0)-np.array(s_obs))

## K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),H)

(1, 2) (1, 2)


4.082727736487477

In [109]:
def abc_algo (N, S, y_obs, H):
    s_obs = S(y_obs)
    theta=[]
    s = []
    y = []
    theta_0 = prior().sample()
    y_0 = single_lik(theta_0).sample() 
    s_0 = S(y_0)
    
    print('prima')
    while ((K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),H)[0]<=np.array([0,0])[0]) and ( K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),H)[1]<=np.array([0,0])[1])):
        print(K_h(np.linalg.norm(s_0-s_obs),H))
        theta_0 = prior().sample()
        y_0 = single_lik(theta_0).sample() 
        s_0 = S(y_0)
    print('dopo')
      
    theta.append(theta_0)
    s.append(s_0)
    y.append(y_0)
    theta_old = theta_0
    s_old = s_0
    y_old = y_0

    for i in range(N):
        theta_current = proposal(theta_old).sample()
        y_current = single_lik(theta_current).sample(1000)
        #y.append(y_current)
        s_current = S(y_current)
               
        prob = min([1.,(K_h(np.linalg.norm(s_current - s_obs),H)@prior().prob(theta_current)@proposal(theta_old).prob(theta_current))/(
             K_h(np.linalg.norm(s_old-s_obs),H)@prior().prob(theta_old)@proposal(theta_current).prob(theta_old))])
        u = np.random.uniform(low=0,high=1,size=2)
        if ((u[0] < prob[0]) and (u[1] < prob[1])):
              theta_i = theta_current
              s_i = s_current
        else: 
              theta_i = theta_old
              s_i = s_old
              #y.pop()
              #y.append(y_old)

        theta_old = theta_i
        s_old = s_i
        
        theta.append(theta_i)
        s.append(s_i)
        
    return  theta

In [45]:
np.random.uniform(low=0,high=1,size=2)

array([0.2362373 , 0.20395123])

In [43]:
prior().prob(proposal(prior().sample()).sample())@proposal(prior().sample()).prob(proposal(prior().sample()).sample())

1.338803565385489e-06

In [100]:
##VARIABILI:
cov= np.array([[1,0],[0,1]])
H= np.array([[1,0],[0,1]])
mu0 = np.array ([[3,6]])
sigma0 = np.array([[3,0.7],[0.7,3]])
n = 1000
mu_obs = np.array ([[1,4]])
sigma_obs=np.array([[1,0.5],[0.5,1]])
y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)

In [110]:
prova = abc_algo (n, S2, y_obs, H) 

prima


ValueError: ignored